In [1]:
filepath = '/Users/jiangwolin/IdeaProjects/scala-test/data/intermediateResults/OD-Matrix'

In [7]:
from empirical_class import ODMatrix
od = ODMatrix(filepath)
od.load_matrix()

In [12]:
from epr import EPR
import pandas as pd 

start_time = pd.to_datetime('2019/01/01 08:00:00')
end_time = pd.to_datetime('2019/01/20 08:00:00')

epr_instance = EPR()
# Input: starting time, ending time, filepath to the empirical OD matrix, number of agents
d = epr_instance.generate(start_time, end_time, filepath, n_agents=10000)

100%|██████████| 10000/10000 [03:40<00:00, 45.44it/s]


In [13]:
from helper_functions import synthetic_trips
import h3
import folium

In [14]:
trips = synthetic_trips(d)
df = trips.groupby(["origin", "destination"]).size().reset_index().sort_values(by=0, ascending=False)
df.rename(columns={0: 'count'}, inplace=True)
df['origin_coords'] = df['origin'].apply(lambda x: h3.cell_to_latlng(x))
df['dest_coords'] = df['destination'].apply(lambda x: h3.cell_to_latlng(x))

# Split the tuples into separate latitude and longitude columns
df[['origin_lat', 'origin_lng']] = pd.DataFrame(df['origin_coords'].tolist(), index=df.index)
df[['dest_lat', 'dest_lng']] = pd.DataFrame(df['dest_coords'].tolist(), index=df.index)
df

,origin,destination,count,origin_coords,dest_coords,origin_lat,origin_lng,dest_lat,dest_lng
28069,8329a1fffffffff,8329a1fffffffff,108976,"(34.500094979532754, -118.44763026078893)","(34.500094979532754, -118.44763026078893)",34.500095,-118.447630,34.500095,-118.447630
27424,8329a0fffffffff,8329a0fffffffff,92084,"(33.969092351434384, -117.24645368593889)","(33.969092351434384, -117.24645368593889)",33.969092,-117.246454,33.969092,-117.246454
24688,832986fffffffff,832986fffffffff,81962,"(36.279440090893914, -114.82034498786267)","(36.279440090893914, -114.82034498786267)",36.279440,-114.820345,36.279440,-114.820345
16690,832830fffffffff,832830fffffffff,73899,"(38.1932089549308, -122.19619675659699)","(38.1932089549308, -122.19619675659699)",38.193209,-122.196197,38.193209,-122.196197
29312,8329a4fffffffff,8329a4fffffffff,46434,"(32.82616396056561, -117.23572206484211)","(32.82616396056561, -117.23572206484211)",32.826164,-117.235722,32.826164,-117.235722
...,...,...,...,...,...,...,...,...,...
30899,8329a8fffffffff,832a8bfffffffff,1,"(36.13580963822775, -119.68898706368451)","(36.14453713567779, -79.64405165437488)",36.135810,-119.688987,36.144537,-79.644052
30894,8329a8fffffffff,832a33fffffffff,1,"(36.13580963822775, -119.68898706368451)","(41.804617423063064, -71.56622819284895)",36.135810,-119.688987,41.804617,-71.566228
4041,832656fffffffff,8326d1fffffffff,1,"(39.196372885191444, -94.44037532634131)","(35.07304462671108, -101.33099490338864)",39.196373,-94.440375,35.073045,-101.330995
30891,8329a8fffffffff,832a1afffffffff,1,"(36.13580963822775, -119.68898706368451)","(38.16426998181517, -74.38680098225775)",36.135810,-119.688987,38.164270,-74.386801


In [23]:
# Parameters for plotting thickness
max_count = df[(df["count"] > 100) & (df["origin"] != df["destination"])]["count"].max() 
print(max_count)
min_thickness = 0.1 #minimum thickness to visualize
max_thickness = 10   #maximum thickness to visualize, corresponding to max_count / 50 * max_thickness 
factor = 10        #thickness = count/max * factor
cutoff_count = 500  # Only visualize movement with count larger than this
max_circle_size = 50

#Initiate flow_counter, for subsequent procedure of adding circles 
unique_destinations = df["destination"].unique()
unique_origins = df["origin"].unique()

# Combine the unique values from both columns into a single array
combined_unique_values = pd.unique(
    pd.concat([pd.Series(unique_destinations), pd.Series(unique_origins)])
)

# Create a Series with combined_unique_values as the index and initialize all values to 0
flow_counter = pd.Series(0, index=combined_unique_values, name="count").astype(float)


map_center = [df['origin_lat'].mean(), df['origin_lng'].mean()]
m = folium.Map(location=map_center, zoom_start=5)

for idx, row in df.iterrows():
    start = (row['origin_lat'], row['origin_lng'])
    end = (row['dest_lat'], row['dest_lng'])
    thickness = max(min(row["count"] / max_count * factor, max_thickness), min_thickness)
    if (row["count"] > cutoff_count) & (start != end):
        flow_counter[row["origin"]] += thickness
        flow_counter[row["destination"]] += thickness
        folium.PolyLine(
            [start, end],
            color="blue",
            weight=thickness,  # Adjust divisor to 
            opacity=0.2
        ).add_to(m)


division = (flow_counter.max()/max_circle_size)

for index, item in flow_counter.items():
    if item > 0.1: 
        folium.CircleMarker(
            location = h3.cell_to_latlng(index),
            radius = item / division,
            weight=0,
            color='red',  # Adjust the color as needed
            fill=True,
            fill_color='red'
        ).add_to(m)
m

14691
